# Build a regression model: linear and polynomial regression models


## Linear and Polynomial Regression for Pumpkin Pricing - Lesson 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic by Dasani Madipalli</figcaption>


<!--![Infographic by Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.en.png){width="800"}-->

#### Introduction

Up to this point, you’ve explored what regression is using sample data from the pumpkin pricing dataset that we’ll be working with throughout this lesson. You’ve also visualized the data using `ggplot2`. 💪

Now, you’re ready to dive deeper into regression for machine learning. In this lesson, you’ll learn more about two types of regression: *basic linear regression* and *polynomial regression*, along with some of the mathematics behind these techniques.

> Throughout this curriculum, we assume minimal math knowledge and aim to make the content accessible to students from various fields. Look out for notes, 🧮 math callouts, diagrams, and other learning tools to help with understanding.

#### Preparation

As a reminder, you’re working with this data to answer specific questions:

-   When is the best time to buy pumpkins?

-   What price can I expect for a case of miniature pumpkins?

-   Should I buy them in half-bushel baskets or in 1 1/9 bushel boxes? Let’s keep exploring this data.

In the previous lesson, you created a `tibble` (a modern reimagining of the data frame) and filled it with part of the original dataset, standardizing the pricing by the bushel. However, by doing so, you were only able to gather about 400 data points, and only for the fall months. Could we uncover more details about the data by cleaning it further? Let’s find out... 🕵️‍♀️

For this task, we’ll need the following packages:

-   `tidyverse`: The [tidyverse](https://www.tidyverse.org/) is a [collection of R packages](https://www.tidyverse.org/packages) designed to make data science faster, easier, and more enjoyable!

-   `tidymodels`: The [tidymodels](https://www.tidymodels.org/) framework is a [collection of packages](https://www.tidymodels.org/packages/) for modeling and machine learning.

-   `janitor`: The [janitor package](https://github.com/sfirke/janitor) provides simple tools for examining and cleaning messy data.

-   `corrplot`: The [corrplot package](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) offers a visual exploratory tool for correlation matrices, with automatic variable reordering to help detect hidden patterns among variables.

You can install them using the following command:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

The script below checks whether you have the required packages for this module and installs them for you if they’re missing.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

We'll later load these awesome packages and make them available in our current R session. (This is just for illustration purposes, `pacman::p_load()` already does this for you)

## 1. A linear regression line

As you learned in Lesson 1, the goal of a linear regression exercise is to plot a *line of best fit* to:

-   **Show variable relationships**. Illustrate the relationship between variables.

-   **Make predictions**. Accurately predict where a new data point might fall in relation to that line.

To draw this type of line, we use a statistical technique called **Least-Squares Regression**. The term `least-squares` means that all the data points surrounding the regression line are squared and then summed. Ideally, this final sum is as small as possible, because we want a low number of errors, or `least-squares`. Therefore, the line of best fit is the line that minimizes the sum of the squared errors—hence the name *least squares regression*.

We do this because we want to model a line that has the smallest cumulative distance from all of our data points. The terms are squared before summing because we care about the magnitude of the errors, not their direction.

> **🧮 Show me the math**
>
> This line, called the *line of best fit*, can be expressed by [an equation](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` is the '`explanatory variable` or `predictor`'. `Y` is the '`dependent variable` or `outcome`'. The slope of the line is `b`, and `a` is the y-intercept, which represents the value of `Y` when `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "slope = $y/x$")
    Infographic by Jen Looper
>
> First, calculate the slope `b`.
>
> In other words, referring to our pumpkin data's original question: "predict the price of a pumpkin per bushel by month," `X` would represent the price, and `Y` would represent the month of sale.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.en.png)
    Infographic by Jen Looper
> 
> Calculate the value of Y. If you're paying around \$4, it must be April!
>
> The math used to calculate the line must account for the slope of the line, which also depends on the intercept, or where `Y` is positioned when `X = 0`.
>
> You can see the method for calculating these values on the [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) website. Also, check out [this Least-Squares Calculator](https://www.mathsisfun.com/data/least-squares-calculator.html) to see how the values affect the line.

Not so intimidating, right? 🤓

#### Correlation

Another important term to understand is the **Correlation Coefficient** between the given X and Y variables. Using a scatterplot, you can quickly visualize this coefficient. A plot where the data points form a neat line has high correlation, while a plot where the data points are scattered randomly between X and Y has low correlation.

A good linear regression model will have a high Correlation Coefficient (closer to 1 than 0) when using the Least-Squares Regression method with a line of regression.


## **2. A dance with data: creating a data frame that will be used for modeling**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Artwork by @allison_horst</figcaption>


<!--![Artwork by \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.en.jpg){width="700"}-->


Load the necessary libraries and dataset. Transform the data into a data frame containing a subset of the data:

-   Only include pumpkins priced by the bushel.

-   Convert the date into a month.

-   Calculate the price as the average of the high and low prices.

-   Adjust the price to reflect the cost per bushel quantity.

> These steps were covered in the [previous lesson](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

In the spirit of sheer adventure, let's explore the [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) that provides simple functions for examining and cleaning dirty data. For instance, let's take a look at the column names for our data:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 We can do better. Let's make these column names `friendR` by converting them to the [snake_case](https://en.wikipedia.org/wiki/Snake_case) convention using `janitor::clean_names`. To find out more about this function: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Much tidyR 🧹! Now, let's dance with the data using `dplyr` just like in the previous lesson! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Good job!👌 You now have a clean, organized dataset ready to build your new regression model!

How about a scatter plot?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


A scatter plot shows that we only have data for the months from August to December. We likely need more data to make conclusions in a linear manner.

Let's revisit our modeling data:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

What if we wanted to predict the `price` of a pumpkin based on the `city` or `package` columns, which are character types? Or, even more simply, how could we determine the correlation (which requires both inputs to be numeric) between, for example, `package` and `price`? 🤷🤷

Machine learning models perform best with numeric features rather than text values, so it's usually necessary to convert categorical features into numeric representations.

This means we need to find a way to reformat our predictors to make them more suitable for a model to use effectively—a process known as `feature engineering`.


## 3. Preprocessing data for modeling with recipes 👩‍🍳👨‍🍳

Activities that reshape predictor values to make them more suitable for a model are often referred to as `feature engineering`.

Different models have different preprocessing needs. For example, least squares requires `encoding categorical variables` like month, variety, and city_name. This involves `converting` a column with `categorical values` into one or more `numeric columns` that replace the original.

For instance, imagine your data includes the following categorical feature:

|  city   |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

You can use *ordinal encoding* to replace each category with a unique integer value, like this:

| city |
|:----:|
|  0   |
|  1   |
|  2   |

And that's exactly what we'll do with our data!

In this section, we'll dive into another fantastic Tidymodels package: [recipes](https://tidymodels.github.io/recipes/) - designed to help you preprocess your data **before** training your model. At its core, a recipe is an object that specifies the steps to be applied to a dataset to prepare it for modeling.

Now, let's create a recipe that gets our data ready for modeling by replacing all the observations in the predictor columns with unique integers:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Awesome! 👏 We just created our first recipe that defines an outcome (price) and its corresponding predictors, ensuring that all predictor columns are encoded as integers 🙌! Let's break it down step by step:

-   The `recipe()` function, combined with a formula, assigns *roles* to the variables using the `new_pumpkins` dataset as a reference. For example, the `price` column is designated as the `outcome`, while the other columns are assigned the `predictor` role.

-   `step_integer(all_predictors(), zero_based = TRUE)` specifies that all predictors should be converted into integers, starting the numbering at 0.

You might be thinking: "This is so cool!! But what if I want to make sure the recipes are doing exactly what I expect them to do? 🤔"

That's a great question! Once your recipe is defined, you can calculate the parameters needed to preprocess the data and then extract the processed data. While this step isn't typically necessary when using Tidymodels (we'll explore the standard approach soon—via `workflows`), it can be useful for performing a sanity check to ensure the recipes are functioning as intended.

To do this, you'll need two additional functions: `prep()` and `bake()`. And, as always, our friendly R illustrations by [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) make it easier to understand!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Artwork by @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): calculates the necessary parameters from a training dataset, which can then be used on other datasets. For example, for a specific predictor column, it determines which observation will be assigned integer values like 0, 1, 2, and so on.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): uses a prepped recipe to apply the transformations to any dataset.

With that in mind, let's prep and bake our recipes to confirm that, behind the scenes, the predictor columns will first be encoded before fitting a model.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo! 🥳 The processed data `baked_pumpkins` has all its predictors encoded, confirming that the preprocessing steps defined in our recipe are working as expected. While this might make it harder for you to read, it becomes much more understandable for Tidymodels! Take a moment to identify which observation has been mapped to its corresponding integer.

It's also worth noting that `baked_pumpkins` is a data frame that we can use for computations.

For example, let's try to identify a strong correlation between two points in your data to potentially build a solid predictive model. We'll use the function `cor()` for this. Type `?cor()` to learn more about the function.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


As it turns out, there's only a weak correlation between the City and Price. However, there's a slightly stronger correlation between the Package and its Price. That makes sense, right? Typically, the larger the produce box, the higher the price.

While we're at it, let's also try visualizing a correlation matrix of all the columns using the `corrplot` package.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Much better.

A good question to ask now about this data is: '`What price can I expect for a given pumpkin package?`' Let's dive right in!

> Note: When you **`bake()`** the prepared recipe **`pumpkins_prep`** with **`new_data = NULL`**, you retrieve the processed (i.e., encoded) training data. If you had another dataset, such as a test set, and wanted to see how the recipe would pre-process it, you would simply bake **`pumpkins_prep`** with **`new_data = test_set`**.

## 4. Build a linear regression model

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic by Dasani Madipalli</figcaption>


Now that we have created a recipe and confirmed that the data will be pre-processed correctly, let's proceed to build a regression model to answer the question: `What price can I expect for a given pumpkin package?`

#### Train a linear regression model using the training set

As you may have already noticed, the column *price* is the `outcome` variable, while the *package* column is the `predictor` variable.

To achieve this, we'll first split the data so that 80% is allocated to the training set and 20% to the test set. Then, we'll define a recipe that encodes the predictor column into a set of integers, followed by building a model specification. We won't prep and bake our recipe since we already know it will preprocess the data as intended.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Great job! Now that we have a recipe and a model specification, we need a way to combine them into a single object that will preprocess the data (prep + bake behind the scenes), fit the model on the preprocessed data, and even allow for potential post-processing steps. Sounds reassuring, right? 🤩

In Tidymodels, this handy object is called a [`workflow`](https://workflows.tidymodels.org/), and it conveniently organizes all your modeling components! This is similar to what we’d refer to as *pipelines* in *Python*.

So, let’s package everything into a workflow! 📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

In addition, a workflow can be adjusted or trained in a similar way to how a model is.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

From the model output, we can see the coefficients learned during training. They represent the coefficients of the line of best fit that minimizes the overall error between the actual and predicted variable.

#### Evaluate model performance using the test set

It's time to check how the model performed 📏! How do we do this?

Now that we've trained the model, we can use it to make predictions for the test_set using `parsnip::predict()`. Afterward, we can compare these predictions to the actual label values to assess how well (or poorly!) the model is functioning.

Let's begin by making predictions for the test set and then binding the columns to the test set.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Yes, you’ve just trained a model and used it to make predictions! 🔮 How good is it? Let’s evaluate the model’s performance!

In Tidymodels, we do this using `yardstick::metrics()`! For linear regression, let’s focus on the following metrics:

-   `Root Mean Square Error (RMSE)`: The square root of the [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). This gives an absolute metric in the same unit as the target variable (in this case, the price of a pumpkin). The smaller the value, the better the model (in simple terms, it represents the average amount by which the predictions are off!).

-   `Coefficient of Determination (commonly known as R-squared or R2)`: A relative metric where a higher value indicates a better fit of the model. Essentially, this metric shows how much of the variance between the predicted and actual target values the model is able to explain.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

There goes the model performance. Let's see if we can get a clearer picture by visualizing a scatter plot of the package and price, then overlaying a line of best fit using the predictions.

To do this, we'll need to prepare and process the test set to encode the package column, and then combine it with the predictions generated by our model.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Great! As you can see, the linear regression model doesn't do a very good job of capturing the relationship between a package and its corresponding price.

🎃 Congratulations, you've just created a model that can help predict the price of several types of pumpkins. Your holiday pumpkin patch will look amazing! But you can probably build an even better model.

## 5. Build a polynomial regression model

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographic by Dasani Madipalli</figcaption>


<!--![Infographic by Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.en.png){width="800"}-->


Sometimes our data might not follow a linear pattern, but we still want to predict an outcome. Polynomial regression can help us make predictions for more complex, non-linear relationships.

For example, consider the relationship between package size and price in our pumpkin dataset. While there might sometimes be a linear relationship between variables—like the larger the pumpkin's volume, the higher the price—there are cases where these relationships can't be represented by a plane or straight line.

> ✅ Here are [some more examples](https://online.stat.psu.edu/stat501/lesson/9/9.8) of data that could benefit from polynomial regression.
>
> Take another look at the relationship between Variety and Price in the previous plot. Does this scatterplot seem like it should necessarily be analyzed with a straight line? Perhaps not. In such cases, you can try polynomial regression.
>
> ✅ Polynomials are mathematical expressions that may include one or more variables and coefficients.

#### Train a polynomial regression model using the training set

Polynomial regression creates a *curved line* to better fit non-linear data.

Let's explore whether a polynomial model can perform better in making predictions. We'll follow a process similar to what we did earlier:

-   Create a recipe that outlines the preprocessing steps needed to prepare our data for modeling, such as encoding predictors and calculating polynomials of degree *n*.

-   Define a model specification.

-   Combine the recipe and model specification into a workflow.

-   Train the model by fitting the workflow.

-   Assess how well the model performs on the test data.

Let's dive in!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evaluate model performance

👏👏You've created a polynomial model—let's make predictions on the test set!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, let's evaluate how the model performed on the test_set using `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Much better performance.

The `rmse` dropped from approximately 7 to around 3, indicating a smaller error between the actual price and the predicted price. You can *roughly* interpret this as meaning that, on average, incorrect predictions are off by about $3. The `rsq` improved from roughly 0.4 to 0.8.

All these metrics show that the polynomial model significantly outperforms the linear model. Great work!

Let’s see if we can visualize this!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


You can see a curved line that better fits your data! 🤩

You can make it even smoother by providing a polynomial formula to `geom_smooth` like this:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Much like a smooth curve!🤩

Here's how you would make a new prediction:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


The `polynomial model` prediction makes sense when you look at the scatter plots of `price` and `package`! And, if this model performs better than the previous one based on the same data, you’ll need to plan for these pricier pumpkins!

🏆 Great job! You’ve built two regression models in one lesson. In the final section on regression, you’ll explore logistic regression to classify categories.

## **🚀Challenge**

Experiment with different variables in this notebook to see how correlation impacts model accuracy.

## [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Review & Self Study**

In this lesson, we covered Linear Regression. There are other significant types of Regression. Look into Stepwise, Ridge, Lasso, and Elasticnet techniques. A great resource to dive deeper is the [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

If you’re interested in learning more about the powerful Tidymodels framework, check out these resources:

-   Tidymodels website: [Get started with Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn and Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **THANK YOU TO:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) for creating the wonderful illustrations that make R more approachable and fun. You can find more of her work in her [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we aim for accuracy, please note that automated translations may include errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is advised. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
